<a href="https://colab.research.google.com/github/grawlithe/conduit-assessment-python/blob/main/Solve_Business_Problems_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solve Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

You are encouraged to use AI assistants (like ChatGPT or Claude) and any IDE of your choice to develop your solution. Many modern IDEs (such as PyCharm, or Cursor) can work with Jupiter files directly.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [12]:
# Install the OpenAI Python package.
%pip install openai httpx==0.27.2

**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [13]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='a0Bfv0000036ye6EAA'
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [14]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [15]:
# Authenticate the user to be able to create new worksheet
# Insert the authentication snippet from the official documentation to create a google client:
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=qzi9VsEqzI-o

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [16]:
# Code example of generating output document

# Creates a new shared Google Worksheet every invocation with the proper structure
# Note: This code should be executed from the google colab once you are ready, it will not work locally
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# IMPORTANT: You need to authenticate the user to be able to create new worksheet
# Insert the authentication snippet from the official documentation to create a google client:
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=qzi9VsEqzI-o

# This code goes after creating google client
output_document = gc.create('Solving Business Problems with AI - Output')

# Create 'email-classification' sheet
email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')

# Example of writing the data into the sheet
# Assuming you have your classification in the email_classification_df DataFrame
# set_with_dataframe(email_classification_sheet, email_classification_df)
# Or directly update cells: https://docs.gspread.org/en/latest/user-guide.html#updating-cells

# Create 'order-status' sheet
order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

# Create 'order-response' sheet
order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Create 'inquiry-response' sheet
inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')

# This is the solution output link, paste it into the submission form
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")

Shareable link: https://docs.google.com/spreadsheets/d/1kVXHdqcqUU1RxiZOc5wQVwLUNWYuqJ6whdu-pOvvxqs


# Task 1. Classify emails

In [17]:
import pandas as pd
from IPython.display import display

# This function classifies an email using the OpenAI API.
def classify_email(email_body):
    """
    Classifies an email as either a 'product inquiry' or an 'order request'
    using the OpenAI gpt-4o model.
    """
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies emails. Please classify the email as either 'product inquiry' or 'order request'. Respond with only the category name."},
                {"role": "user", "content": email_body}
            ]
        )
        # The response from the model is extracted.
        return completion.choices[0].message.content.strip()
    except Exception as e:
        # Errors in API calls are caught and reported.
        print(f"An error occurred: {e}")
        return "classification_error"

# A list to hold the classification results.
email_classifications = []

# Each email from the emails DataFrame is processed.
for index, row in emails_df.iterrows():
    # The body of the email is used for classification.
    classification = classify_email(row['message'])
    # The result is stored along with the email ID.
    email_classifications.append({'email ID': row['email_id'], 'category': classification})

# The results are converted into a pandas DataFrame.
email_classification_df = pd.DataFrame(email_classifications)

# The first 5 rows of the resulting DataFrame are displayed for review.
display(email_classification_df.head())

,email ID,category
0,E001,Order request
1,E002,order request
2,E003,product inquiry
3,E004,order request
4,E005,product inquiry


# Task 2. Process order requests

In [18]:
import json
import pandas as pd
from IPython.display import display

# Create a copy of the products dataframe to manage inventory.
products_inventory_df = products_df.copy()

def parse_order_request(email_body, products_df):
    """
    Parses an email to extract order details using an LLM.
    Returns a list of dictionaries with product_id and quantity.
    """
    # Create a simplified catalog for the prompt to help the LLM.
    product_list_for_prompt = products_df[['product_id', 'name']].to_string(index=False)

    prompt = f"""
    You are an order processing assistant. Parse the following email to extract the product(s) and quantity being ordered.
    Respond with a JSON array of objects, where each object has "product_id" and "quantity".
    The "product_id" MUST be one of the IDs from the product list below. Match the customer's request to the closest product.
    If a quantity is not specified for a product, assume it is 1.

    Product List:
    {product_list_for_prompt}

    Email:
    "{email_body}"

    JSON Output:
    """
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert order processing assistant that returns JSON."},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}
        )
        # The response from the model is extracted and parsed as JSON.
        response_text = completion.choices[0].message.content
        # The model might return a JSON object with a key, so we look for the first list.
        response_json = json.loads(response_text)
        # Find the first list in the JSON response, which should be the order items.
        for key, value in response_json.items():
            if isinstance(value, list):
                return value
        return [] # Return empty list if no list is found
    except Exception as e:
        print(f"An error occurred during order parsing: {e}")
        return []

def generate_order_response(order_results):
    """
    Generates a customer-facing email response based on the order processing results.
    """
    prompt = f"""
    You are a customer service representative for a fashion store.
    Write a professional and friendly email response to a customer based on the following order processing results.

    - If the order is fully processed (all items 'created'), confirm the order and thank the customer.
    - If some items are 'out of stock', inform the customer which items are unavailable and apologize for the inconvenience. Let them know you'll notify them when the items are back in stock.
    - If all items are 'out of stock', express regret and offer to notify them about restocking.

    Order Processing Results:
    {json.dumps(order_results, indent=2)}

    Email Response:
    """
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful customer service assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred during response generation: {e}")
        return "Could not generate a response at this time. Please contact support."

# Lists to hold the results.
order_status_list = []
order_response_list = []

# Filter for emails classified as 'order request'.
order_request_emails = email_classification_df[email_classification_df['category'] == 'order request']
order_request_emails = pd.merge(order_request_emails, emails_df, left_on='email ID', right_on='email_id')

# Process each order request.
for index, row in order_request_emails.iterrows():
    email_id = row['email ID']
    email_body = row['message']

    # Use the LLM to parse the order from the email.
    ordered_items = parse_order_request(email_body, products_inventory_df)

    if not ordered_items:
        continue

    processed_items_for_response = []

    # Process each item in the order.
    for item in ordered_items:
        product_id = item.get('product_id')
        quantity = int(item.get('quantity', 1))

        # Find the product in our inventory.
        product_info = products_inventory_df[products_inventory_df['product_id'] == product_id]

        if not product_info.empty:
            stock_idx = product_info.index[0]
            stock_amount = products_inventory_df.loc[stock_idx, 'stock']
            product_name = products_inventory_df.loc[stock_idx, 'name']

            if stock_amount >= quantity:
                # Item is in stock.
                products_inventory_df.loc[stock_idx, 'stock'] -= quantity
                status = 'created'
            else:
                # Item is out of stock.
                status = 'out of stock'

            order_status_list.append({'email ID': email_id, 'product ID': product_id, 'quantity': quantity, 'status': status})
            processed_items_for_response.append({'product_name': product_name, 'quantity': quantity, 'status': status})
        else:
            # Product ID not found in catalog.
            order_status_list.append({'email ID': email_id, 'product ID': product_id, 'quantity': quantity, 'status': 'product_not_found'})
            processed_items_for_response.append({'product_name': f"Unknown Product (ID: {product_id})", 'quantity': quantity, 'status': 'product_not_found'})


    # Generate the email response after processing all items for the current email.
    response_text = generate_order_response(processed_items_for_response)
    order_response_list.append({'email ID': email_id, 'response': response_text})

# Create DataFrames from the results.
order_status_df = pd.DataFrame(order_status_list)
order_response_df = pd.DataFrame(order_response_list)

# Display the first 5 rows of each new DataFrame.
print("Order Status Sheet:")
display(order_status_df.head())
print("\nOrder Response Sheet:")
display(order_response_df.head())

Order Status Sheet:


,email ID,product ID,quantity,status
0,E002,VBT2345,1,created
1,E007,CLF2109,5,out of stock
2,E007,FZZ1098,2,created



Order Response Sheet:


,email ID,response
0,E002,Subject: Order Confirmation - Thank You for Sh...
1,E007,Subject: Update on Your Recent Order with Us\n...


# Task 3. Handle product inquiry